Comparison of different c2l models for NanostringWTA data:

In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
from IPython.display import Image
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True' + ',dnn.enabled=False'
import pycell2location
from pycell2location import models

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Can not use cuDNN on context None: Disabled by dnn.enabled flag
Mapped name None to device cuda: Tesla V100-SXM2-32GB (0000:62:00.0)


ModuleNotFoundError: No module named 'cell2location.models.base'

Load data and subset to one 19pcw replicate:

In [ ]:
adata_wta = pickle.load(open('/nfs/team283/aa16/KR_NAS/data/nanostringWTA_fetailBrain_AnnData.p', 'rb'))
adata_wta = adata_wta[adata_wta.obs['slide'] == '00MU',:]

In [ ]:
meanExpression_sc = pd.read_csv("/nfs/team283/aa16/KR_NAS/data/polioudakis2019_meanExpressionProfiles.csv", index_col=0)

In [ ]:
counts_negativeProbes = np.asarray(adata_wta[:,np.array(adata_wta.var_names =='NegProbe-WTX').squeeze()].X)
counts_nuclei = np.asarray(adata_wta.obs['nuclei']).reshape(len(adata_wta.obs['nuclei']),1)
adata_wta = adata_wta[:,np.array(adata_wta.var_names != 'NegProbe-WTX').squeeze()]

In [ ]:
cell2location.run_c2l.run_cell2location(meanExpression_sc, adata_wta, 
        model_name=pycell2location.models.LocationModelLinearDependentWMultiExperimentLocationBackgroundNormGeneAlpha,
        train_args={'use_raw': False},
        model_kwargs={
        "cell_number_prior" : {'cells_per_spot': counts_nuclei, 'factors_per_spot': 6, 'combs_per_spot': 3},
        "cell_number_var_prior" : {'cells_mean_var_ratio': 1, 'factors_mean_var_ratio': 1, 'combs_mean_var_ratio': 1}})

In [ ]:
pycell2location